# Problema di Orienteering per il tour degli Scavi di Pompei

### Descrizione del problema 

<justify> L'orienteering è uno sport all'aria aperta solitamente praticato in una zona montuosa o ricca di foreste. Armati di bussola e mappa, i concorrenti partono da un punto di controllo specificato, cercano di visitare il maggior numero possibile di altri punti di controllo entro un limite di tempo prescritto e tornano al punto di controllo specificato.
    
- Ogni punto di controllo ha un punteggio associato, in modo che l'obiettivo dell'orienteering sia massimizzare il punteggio totale.
- I concorrenti che arrivano al traguardo dopo che il tempo è scaduto sono squalificati e il concorrente idoneo con il punteggio più alto viene dichiarato vincitore.
- Poiché il tempo è limitato, i concorrenti potrebbero non essere in grado di visitare tutti i punti di controllo. I concorrenti devono selezionare un sottoinsieme di punti di controllo da visitare che massimizzeranno il loro punteggio totale soggetto alla restrizione temporale.

Poiché la distanza e il tempo di percorrenza tra qualsiasi coppia di punti di controllo sono determinati dalla geografia, assumeremo che si tratti di quantità note. Pertanto, il problema può essere formulato nel modo seguente.
Dati n nodi nel piano euclideo ciascuno con un punteggio s_i≥0 (si noti che s_1 = 0) trovano un percorso di punteggio massimo attraverso questi nodi che inizia e termina al punto di controllo (1) di lunghezza (o durata) non superiore a TMAX. </justify>

### Formulazione del problema

- **Variabili decisionali:**

    - $x_{ij}$ = 1 se l’arco (𝑖,𝑗) ∈ 𝐴 appartiene al percorso seguito dal concorrente, 0 altrimenti

    - $𝑦_𝑖$ = 1  se il nodo 𝑖∈𝑉 è visitato, 0 altrimenti
    
    - $u_i \rightarrow$  ordine di visita del vertice i nella soluzione (𝑖∈𝑉)

- **Funzione Obiettivo:**

\begin{equation}
    max \sum_{i=1}^n s_i y_i
\end{equation}

- **Vincoli:**

\begin{equation}
    \sum_{j=2}^n x_{1j} = \sum_{i=1}^{n-1} x_{in} = 1
\end{equation}
\begin{equation}
    \sum_{i=1 \\ i \neq j}^{n} x_{ij} = \sum_{i=1 \\ i \neq j}^{n} x_{ji} \hspace{1cm} j = 2..n-1
\end{equation}
\begin{equation}
    \sum_{i=1 \\ i \neq j}^n x_{ij} = y_j \hspace{1cm} j = 1..n
\end{equation}
\begin{equation}
    \sum_{i=1}^n \sum_{j=1 \\ i \neq j}^n d_{ij} x_{ij} \leq TMax
\end{equation}
\begin{equation}
    u_1 = 0
\end{equation}
\begin{equation}
    0 \leq u_i \leq n-1 \hspace{1cm} intera \hspace{1cm} i=1..n
\end{equation}
\begin{equation}
    u_j - u_i \geq 1-n(1-x_{ij}) \hspace{1cm} i,j = 1..n,i \neq j,j \neq 1
\end{equation}

### Lettura dati dal .csv

In [1]:
import pandas as pd
import numpy as np

np.random.seed(54)

col_list = ["DESCRIZIONE", "LONGITUDINE", "LATITUDINE"]
df = pd.read_csv('Scavi di Pompei-Siti archeologici.csv', usecols=col_list)

siti = []
coordinate = {}
score = {}

for i in range(len(df)):
    sito = df['DESCRIZIONE'][i]
    siti.append(sito)
    #Si prende prima la latitudide e poi la longitudine per via dell'utilizzo di Folium
    coordinate[sito] = (float(df['LATITUDINE'][i]), float(df['LONGITUDINE'][i])) 
    
    score[sito] = np.random.randint(1,50) #lo score sarà FISSO

### Calcolo la distanza euclidea

In [2]:
import math
from itertools import combinations #<----A CHE COSA SERVE?

def distance(sito1, sito2):
    c1 = coordinate[sito1]
    c2 = coordinate[sito2]
    
    #Metrica di Euclide per la valutazione della distanza tra due punti
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return 100*math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1 in siti for c2 in siti if c1 != c2 }

### Calcolo del punto medio tra quelli dati per rappresentarli sulla mappa

In [3]:
media_lat=0
media_long=0

for sito in siti:
    media_lat = media_lat + coordinate[sito][0]
    media_long = media_long + coordinate[sito][1]
    
lat=media_lat/54
long=media_long/54

In [4]:
#Marker sulla mappa: I nodi del problema
import folium
map = folium.Map(location=[lat,long], zoom_start = 15)
for sito in siti:
    folium.Marker(location = coordinate[sito], tooltip = sito).add_to(map)
map

### Importa la libreria GRB inizializza il problema e definisce le variabili

In [5]:
#MODELLAZIONE GUROBI
import gurobipy as gp
from gurobipy import GRB

import numpy as np

model = gp.Model('Pompei_orienteering') #definizione del modello

#Definizione variabili di decisione
Xvars = model.addVars(dist.keys(), obj = dist, vtype = GRB.BINARY, name = 'x')
Yvars = model.addVars(siti, obj = 0.0, vtype = GRB.BINARY, name = 'y')
#Uvars = model.addVars(siti, obj = 0.0, vtype = GRB.INTEGER, lb=0, ub=len(siti), name = 'u')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-10


### Funzione obiettivo

In [6]:
#Definizione della funzione obiettivo
obj = model.setObjective(gp.quicksum(Yvars[i]*score[i] for i in siti), gp.GRB.MAXIMIZE)

### Vincoli

In [7]:
#Dal nodo di partenza (si indichi quale) esce soltanto un nodo
OutFirst = model.addConstr(Xvars.sum('Villa dei Misteri','*') == 1) 

In [8]:
#dal nodo di uscita (si indichi quale) esce soltanto un nodo
InLast = model.addConstr(Xvars.sum('*','Piazza Esedra') == 1)

In [9]:
#Vincolo del ciclo Hamiltoniamo
Balance = model.addConstrs((gp.quicksum(Xvars.sum(i,j) for i in siti) 
                            == gp.quicksum(Xvars.sum(j,i) for i in siti)
                            for j in siti if i != j and j != 'Piazza Anfiteatro' and j != 'Villa dei Misteri'))

In [10]:
#Vincolo di visita
Visited = model.addConstrs((gp.quicksum(Xvars.sum(j,i) for i in siti) == Yvars[j]
                            for j in siti if i != j))

In [11]:
#Scelta del TMAX (si indica il tempo di visita)
TMAX=6 #limite orario della visita 
MaxTempo = model.addConstr((gp.quicksum(Xvars[i,j]*dist[i,j] for i in siti for j in siti if i != j) <= 0.6*TMAX)) #assumo che il turista percorra gli scavi alla velocità di 0.6km/h

In [12]:
#Vincoli di sottogiro MTZ
#PosizioneINI = model.addConstr(Uvars['Piazza Anfiteatro'] == 1)
#Posizione = model.addConstrs(Uvars[j] - Uvars[i] - len(siti)*Xvars[i,j] >= 1 - len(siti)
                            #for i in siti for j in siti if i != j and j != 'Piazza Anfiteatro')

In [13]:
#Funzioni per eliminare i subtour
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # preleva la soluzione corrente
        Xvals = model.cbGetSolution(model._Xvars)
        selected = gp.tuplelist((i,j) for i, j in model._Xvars.keys() if Xvals[i,j] > 0.5)
        # cerca il ciclo di lunghezza minima nella soluzione
        tour = subtour(selected)
        if len(tour) < len(siti):
            # aggiunge il vincolo di eliminazione di sottogiro
            model.cbLazy(gp.quicksum(model._Xvars[i,j] for i in tour for j in tour if i != j )
                         <= len(tour)-1)

def subtour(edges):
    unvisited = list(range(len(siti)))
    cycle = list(range(len(siti)))
    while unvisited:  # vero se ci sono nodi non visitati
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nuovo percorso più breve
    return cycle

### Scrivo il modello e lo faccio risolvere da Gurobi

In [14]:
model.write('pompei_orienteering.lp')

In [15]:
model.optimize()

model._Xvars = Xvars
model.Params.lazyConstraints = 1
model.optimize(subtourelim)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 109 rows, 2916 columns and 11396 nonzeros
Model fingerprint: 0xcebe2506
Variable types: 0 continuous, 2916 integer (2916 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 54 columns
Presolve time: 0.01s
Presolved: 107 rows, 2862 columns, 8439 nonzeros
Variable types: 0 continuous, 2862 integer (2862 binary)
Found heuristic solution: objective 106.0000000
Found heuristic solution: objective 157.0000000

Root relaxation: objective 9.721021e+02, 280 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  972.10213    0    8  157.00000  972.10213

In [16]:
#Stampa dei risultati
if model.status == GRB.OPTIMAL:

    foundOptimalSol = True
    Xvals = model.getAttr('x', Xvars)
    #Uvals = model.getAttr('x', Uvars)
    solution = gp.tuplelist((i,j) for i,j in Xvals.keys() if Xvals[i,j] > 0.5)

    print('Score ottenuto: %g' % model.objVal)
    print("Lunghezza percorso: %g" % len(solution))
#Stampa 
solution

Score ottenuto: 970
Lunghezza percorso: 46


<gurobi.tuplelist (46 tuples, 2 values each):
 ( Piazza Anfiteatro                        , Anfiteatro di Pompei                     )
 ( Anfiteatro di Pompei                     , Piazza Anfiteatro                        )
 ( Porta Nocera                             , Orto dei Fuggiaschi                      )
 ( Orto dei Fuggiaschi                      , Porta Nocera                             )
 ( Casa della Nave Europa                   , Casa del Triclinio all'aperto            )
 ( Casa del Triclinio all'aperto            , Casa della Nave Europa                   )
 ( Casa del Foro Boario                     , Porta Sarno                              )
 ( Porta Sarno                              , Casa del Foro Boario                     )
 ( Casa di Ottavio Quartione                , Casa del Moralista                       )
 ( Casa del Moralista                       , Casa di Ottavio Quartione                )
 ( Casa dell'Efebo                          , Casa e Thermopoliu

### Stampo il tour degli scavi di Pompei in linea d'aria e via percorribile a piedi

In [17]:
# Stampa del tour degli scavi di Pompei in linea d'aria 
sortedPos = dict(sorted(solution, key = lambda keys: coordinate.keys()))

import folium
map = folium.Map(location=[lat,long], zoom_start = 16)
for sito in sortedPos:
    folium.Marker(location = coordinate[sito], tooltip = sito).add_to(map)

points = []
for sito in sortedPos:
  points.append(coordinate[sito])
#points.append(points[0])

folium.PolyLine(points).add_to(map)

map


In [18]:
#sortedPos = dict(sorted(solution, key = lambda keys: coordinate.keys()))
conv_coord = []
mytour=[]
i=0
for sito in sortedPos:
    conv_coord.append((coordinate[sito][1],coordinate[sito][0]))
    mytour.append(list(conv_coord[i]))
    i+=1

In [19]:
#Stampa del percorso per vie REALI
import openrouteservice as ors
import folium

# API Key di Open Route Service
ors_key = '5b3ce3597851110001cf6248435cfcfbcf0c42858fde19dccf6f9c0f'

# Richiesta dei servizi tramite API Key di ORS
# Apro un Client per effettuare le richieste al Server di ORS
client = ors.Client(key=ors_key)

# Traccio il percorso
route = client.directions(coordinates=mytour,
                          profile='foot-walking',
                          format='geojson')

#SERVIREBBE AGGIUNGERE DUE MARKER DI DUE COLORI DIFFERENTI PER SEGNALARE LO STARTING POINT E L'END POINT
#OCCHIO - SULLA MAPPA CI SONO DUE AMBIENTI NON COPERTI DALLA STRADA (CON 7 ORE)
#OH NO, ANCHE CON SEI ORE SE FACCIO VILLA DEI MISTERI PIAZZA ESEDRA OH NOOO (ABBASSARE LO SCORE DI QUESTI DUE - vai vai)
map = folium.Map(location=[lat,long], zoom_start = 16)
for sito in sortedPos:
    folium.Marker(location = coordinate[sito], tooltip = sito).add_to(map)

# Aggiungo il GeoJson alla mappa
folium.GeoJson(route, name=('Itinerario Scavi di Pompei con ' + str(TMAX) + ' ore')).add_to(map)
    
# Aggiungo il livello del percorso alla mappa
folium.LayerControl().add_to(map)

map

In [20]:
model.write('pompei_orienteering_sol.lp')